# IoT based gesture recognition 
## Gesture based home automation

This notebook is a poc for gesture based home automation - controlling the various appliances at home with gestures.

Gestures are mapped to the corresponding accelerometer and gyroscope values recorded during motion. Here, we have accelerometer and gyroscope values along x, y, z axes recorded 100 times for one gesture, i.e, 600 data points for one gesture.

Dataset: The sensor values were recorded using an app (github link [here](https://github.com/AyishaR/Sensor_data_app)) and consolidates using csv.

Algorithm: A tensorflow deep learning model with relu and softmax activations was used.

![gesture](https://images.unsplash.com/photo-1574676122993-cd7fcf7f9030?ixlib=rb-1.2.1&auto=format&fit=crop&w=500&q=60)

<span>Photo by <a href="https://unsplash.com/@adriensking?utm_source=unsplash&amp;utm_medium=referral&amp;utm_content=creditCopyText">Adrien King</a> on <a href="https://unsplash.com/s/photos/gesture?utm_source=unsplash&amp;utm_medium=referral&amp;utm_content=creditCopyText">Unsplash</a></span>

In [31]:
import pandas as pd
import csv
import os
import tensorflow as tf
import keras
from sklearn.preprocessing import normalize, OneHotEncoder
import numpy as np
import urllib

List of gestures and the various categories for reference. 

Here, count refers to moving to the next iinstance of the appliance (eg, countIncrease - moving from fan1 to fan2, countDecrease - moving from fan2 to fan1) and next refers to choosing the next appliance.

In [4]:
gesture_key = {}
gesture_key["appliance"] = ["light", "fan"]
gesture_key["operation"] = ["switch", "increase", "decrease"]
gesture_key["count"] = ["countIncrease", "countDecrease"]
gesture_key["next"] = ["next"]


gestures = []
for x in gesture_key:
    gestures.extend(gesture_key[x])

print(gestures)

['light', 'fan', 'switch', 'increase', 'decrease', 'countIncrease', 'countDecrease', 'next']


Creating .csv from the .txt obtained from the app.

In [5]:
'''
#gestures = []
with open('/content/gesture25.csv', 'w') as csvfile:
    csvwriter = csv.writer(csvfile)
    for fn in os.listdir('/content/gesture'):
        print(fn)
        #gestures.append(fn[:-4])
        f = open('/content/gesture/' + fn)
        lines = f.readlines()
        print(len(lines))
        for line in lines:
            line = line[:-1]
            csvwriter.writerow(line.split(' '))
#print(gestures)

d25 = pd.read_csv('/content/gesture25.csv')

d25 = d25[d25['Gesture'] != 'Gesture'].reset_index().drop('index', axis = 'columns')
print(d25)
'''

"\n#gestures = []\nwith open('/content/gesture25.csv', 'w') as csvfile:\n    csvwriter = csv.writer(csvfile)\n    for fn in os.listdir('/content/gesture'):\n        print(fn)\n        #gestures.append(fn[:-4])\n        f = open('/content/gesture/' + fn)\n        lines = f.readlines()\n        print(len(lines))\n        for line in lines:\n            line = line[:-1]\n            csvwriter.writerow(line.split(' '))\n#print(gestures)\n\nd25 = pd.read_csv('/content/gesture25.csv')\n\nd25 = d25[d25['Gesture'] != 'Gesture'].reset_index().drop('index', axis = 'columns')\nprint(d25)\n"

In [6]:
# d25.describe()

In [7]:
# d25.to_csv('gesture25_8.csv', index = False)

In [8]:
d25 = pd.read_csv('https://cainvas-static.s3.amazonaws.com/media/user_data/AyishaR0/gesture25_8.csv')

Normalize and shuffle

In [9]:
d25[d25.columns[:-1]] = normalize(d25[d25.columns[:-1]])
d25 = d25.sample(frac=1, random_state=13).reset_index(drop = True)
d25

,1,2,3,4,5,6,7,8,9,10,...,592,593,594,595,596,597,598,599,600,Gesture
0,0.003996,0.091278,0.006338,0.000479,0.000279,-0.000164,0.003996,0.091278,0.006338,-0.000263,...,0.018085,0.001041,-0.000090,0.002073,0.089010,-0.014399,0.018085,0.001041,-0.000090,next
1,-0.004165,0.097079,0.009133,0.000276,-0.000220,0.000453,-0.006155,0.097067,0.008706,0.000276,...,0.006513,-0.001767,0.001087,-0.004082,0.088917,-0.003625,0.006513,-0.001767,0.001087,fan
2,-0.005052,0.091285,0.024677,-0.000003,0.000068,-0.000066,-0.004562,0.090852,0.027865,-0.000003,...,-0.008472,0.001323,0.000837,0.015257,0.120610,0.023229,-0.008472,0.001323,0.000837,decrease
3,0.014024,0.085903,0.020069,-0.000322,-0.000060,-0.000067,0.013478,0.086547,0.020888,-0.000322,...,-0.001762,-0.002843,-0.022217,-0.001339,0.085728,0.027690,-0.000916,-0.000459,-0.003308,switch
4,-0.001346,0.091701,0.018121,-0.000295,-0.000191,-0.000123,-0.001933,0.092476,0.016994,-0.000295,...,-0.002341,0.002211,-0.007130,-0.015979,0.110527,0.028538,-0.002341,0.002211,-0.007130,increase
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206,0.000927,0.088625,0.042406,0.000274,-0.000214,-0.000162,0.001509,0.088831,0.043449,0.000274,...,0.003270,0.000536,0.002506,0.017266,0.095163,-0.008224,0.003270,0.000536,0.002506,light
207,-0.014342,0.094729,0.017700,-0.000006,0.000361,-0.000441,-0.013791,0.094202,0.016167,-0.000006,...,-0.000296,0.000421,-0.000299,0.016794,0.092322,0.030969,-0.000296,0.000421,-0.000299,countDecrease
208,0.001253,0.093697,-0.001351,0.000096,0.000150,-0.000098,0.001854,0.093515,-0.001441,0.000096,...,0.028883,0.002455,-0.000296,-0.002685,0.079184,-0.006945,0.028883,0.002455,-0.000296,next
209,0.007900,0.085158,0.017688,0.000243,-0.000052,0.000032,0.007900,0.085158,0.017688,-0.000025,...,-0.004863,0.001682,-0.016601,-0.008686,0.087935,0.020708,-0.004863,0.001682,-0.016601,switch


One hot encoding the labels

In [10]:
y_df = pd.get_dummies(d25.Gesture)[gestures]
print(y_df)
print(np.array(y_df)[0].shape)

     light  fan  switch  increase  decrease  countIncrease  countDecrease  \
0        0    0       0         0         0              0              0   
1        0    1       0         0         0              0              0   
2        0    0       0         0         1              0              0   
3        0    0       1         0         0              0              0   
4        0    0       0         1         0              0              0   
..     ...  ...     ...       ...       ...            ...            ...   
206      1    0       0         0         0              0              0   
207      0    0       0         0         0              0              1   
208      0    0       0         0         0              0              0   
209      0    0       1         0         0              0              0   
210      1    0       0         0         0              0              0   

     next  
0       1  
1       0  
2       0  
3       0  
4       0  
.. 

In [11]:
df = pd.concat([d25, y_df], axis=1)
df.head()

,1,2,3,4,5,6,7,8,9,10,...,600,Gesture,light,fan,switch,increase,decrease,countIncrease,countDecrease,next
0,0.003996,0.091278,0.006338,0.000479,0.000279,-0.000164,0.003996,0.091278,0.006338,-0.000263,...,-0.000090,next,0,0,0,0,0,0,0,1
1,-0.004165,0.097079,0.009133,0.000276,-0.000220,0.000453,-0.006155,0.097067,0.008706,0.000276,...,0.001087,fan,0,1,0,0,0,0,0,0
2,-0.005052,0.091285,0.024677,-0.000003,0.000068,-0.000066,-0.004562,0.090852,0.027865,-0.000003,...,0.000837,decrease,0,0,0,0,1,0,0,0
3,0.014024,0.085903,0.020069,-0.000322,-0.000060,-0.000067,0.013478,0.086547,0.020888,-0.000322,...,-0.003308,switch,0,0,1,0,0,0,0,0
4,-0.001346,0.091701,0.018121,-0.000295,-0.000191,-0.000123,-0.001933,0.092476,0.016994,-0.000295,...,-0.007130,increase,0,0,0,1,0,0,0,0


In [12]:
train_count = int(0.8*len(df))
test_count = len(df) - train_count
print(train_count, test_count)

x_train, y_train = np.array(df)[:train_count, :600].astype('float32'), np.array(df)[:train_count, -(len(gestures)):].astype('int64')
x_test, y_test = np.array(df)[train_count:, :600].astype('float32'), np.array(df)[train_count:, -(len(gestures)):].astype('int64')

print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

168 43
(168, 600) (168, 8) (43, 600) (43, 8)


In [13]:
x_train[0].shape

(600,)

# Model

In [14]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(512, activation="relu", input_shape = (600,)))
model.add(tf.keras.layers.Dense(128, activation="relu"))
model.add(tf.keras.layers.Dense(64, activation="relu"))
model.add(tf.keras.layers.Dense(len(gestures), activation="softmax"))

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               307712    
_________________________________________________________________
dense_1 (Dense)              (None, 128)               65664     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 8)                 520       
Total params: 382,152
Trainable params: 382,152
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [17]:
model.fit(x_train, y_train, epochs = 16, batch_size = 8)

Epoch 1/16
21/21 [==============================] - 0s 2ms/step - loss: 1.9251 - accuracy: 0.5119
Epoch 2/16
21/21 [==============================] - 0s 2ms/step - loss: 1.1868 - accuracy: 0.7202
Epoch 3/16
21/21 [==============================] - 0s 2ms/step - loss: 0.5560 - accuracy: 0.9226
Epoch 4/16
21/21 [==============================] - 0s 2ms/step - loss: 0.3166 - accuracy: 0.8988
Epoch 5/16
21/21 [==============================] - 0s 2ms/step - loss: 0.1705 - accuracy: 0.9762
Epoch 6/16
21/21 [==============================] - 0s 2ms/step - loss: 0.1025 - accuracy: 0.9881
Epoch 7/16
21/21 [==============================] - 0s 2ms/step - loss: 0.0716 - accuracy: 0.9881
Epoch 8/16
21/21 [==============================] - 0s 2ms/step - loss: 0.0459 - accuracy: 0.9940
Epoch 9/16
21/21 [==============================] - 0s 2ms/step - loss: 0.0305 - accuracy: 0.9940
Epoch 10/16
21/21 [==============================] - 0s 2ms/step - loss: 0.0254 - accuracy: 0.9940
Epoch 11/16
21/21 [

In [18]:
model.evaluate(x_test, y_test)

2/2 [==============================] - 0s 1ms/step - loss: 0.0037 - accuracy: 1.0000


[0.003690351266413927, 1.0]

# Tiny model

In [21]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(16, activation="relu", input_shape = (600,)))
model.add(tf.keras.layers.Dense(8, activation="relu"))
model.add(tf.keras.layers.Dense(4, activation="relu"))
model.add(tf.keras.layers.Dense(len(gestures), activation="softmax"))

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 16)                9616      
_________________________________________________________________
dense_9 (Dense)              (None, 8)                 136       
_________________________________________________________________
dense_10 (Dense)             (None, 4)                 36        
_________________________________________________________________
dense_11 (Dense)             (None, 8)                 40        
Total params: 9,828
Trainable params: 9,828
Non-trainable params: 0
_________________________________________________________________


In [22]:
model.fit(x_train, y_train, epochs = 256, batch_size = 16)
print("\n Test set")
model.evaluate(x_test, y_test)

Epoch 1/256
11/11 [==============================] - 0s 992us/step - loss: 2.0751 - accuracy: 0.1607
Epoch 2/256
11/11 [==============================] - 0s 917us/step - loss: 2.0593 - accuracy: 0.1964
Epoch 3/256
11/11 [==============================] - 0s 934us/step - loss: 2.0428 - accuracy: 0.2381
Epoch 4/256
11/11 [==============================] - 0s 962us/step - loss: 2.0202 - accuracy: 0.2619
Epoch 5/256
11/11 [==============================] - 0s 951us/step - loss: 1.9914 - accuracy: 0.2619
Epoch 6/256
11/11 [==============================] - 0s 1ms/step - loss: 1.9567 - accuracy: 0.2679
Epoch 7/256
11/11 [==============================] - 0s 967us/step - loss: 1.9177 - accuracy: 0.3274
Epoch 8/256
11/11 [==============================] - 0s 980us/step - loss: 1.8722 - accuracy: 0.4762
Epoch 9/256
11/11 [==============================] - 0s 972us/step - loss: 1.8234 - accuracy: 0.4464
Epoch 10/256
11/11 [==============================] - 0s 958us/step - loss: 1.7662 - accuracy

[0.1190461590886116, 0.9767441749572754]

In [23]:
y_pred = model.predict(x_test)

for i in range(len(y_pred)):
    m = max(y_pred[i])
    j = np.where(y_pred[i] == m)[0][0]
    k = np.where(y_test[i] == 1)[0][0]
    if(j!=k):
        print("------- Mismatch -------")
    print("Actual: " + gestures[k] + "\nPredicted: " + gestures[j] + "\nConfidence: " + str(m) + "\n")

Actual: increase
Predicted: increase
Confidence: 0.49649695

Actual: next
Predicted: next
Confidence: 0.9988165

Actual: fan
Predicted: fan
Confidence: 0.9617699

Actual: decrease
Predicted: decrease
Confidence: 0.99917245

Actual: countIncrease
Predicted: countIncrease
Confidence: 0.9992507

Actual: light
Predicted: light
Confidence: 0.981222

------- Mismatch -------
Actual: fan
Predicted: light
Confidence: 0.6490519

Actual: fan
Predicted: fan
Confidence: 0.8963549

Actual: increase
Predicted: increase
Confidence: 0.8386702

Actual: fan
Predicted: fan
Confidence: 0.9944003

Actual: countIncrease
Predicted: countIncrease
Confidence: 0.9935237

Actual: countDecrease
Predicted: countDecrease
Confidence: 0.8907633

Actual: next
Predicted: next
Confidence: 0.9998485

Actual: next
Predicted: next
Confidence: 0.9974916

Actual: countIncrease
Predicted: countIncrease
Confidence: 0.9545595

Actual: countDecrease
Predicted: countDecrease
Confidence: 0.9957747

Actual: increase
Predicted: incr

In [27]:
#model.save('gestureTinyModel.h5')

In [28]:
modelload = tf.keras.models.load_model('gestureTinyModel.h5')

In [29]:
modelload.evaluate(x_test, y_test)

2/2 [==============================] - 0s 1ms/step - loss: 0.0057 - accuracy: 1.0000


[0.0057403589598834515, 1.0]

In [16]:
!deepCC iot/gestureTinyModel.h5

-std=c++11 -O3
reading [keras model] from 'iot/gestureTinyModel.h5'
Saved 'gestureTinyModel.onnx'
reading onnx model from file  gestureTinyModel.onnx
Model info:
  ir_vesion :  3 
  doc       : 
WARN (ONNX): terminal (input/output) dense_8_input's shape is less than 1.
             changing it to 1.
WARN (ONNX): terminal (input/output) dense_11's shape is less than 1.
             changing it to 1.
WARN (GRAPH): found operator node with the same name (dense_11) as io node.
running DNNC graph sanity check ... passed.
Writing C++ file  gestureTinyModel_deepC/gestureTinyModel.cpp
INFO (ONNX): model files are ready in dir gestureTinyModel_deepC
g++ -O3 -I. -I/opt/tljh/user/lib/python3.7/site-packages/deepC-0.13-py3.7-linux-x86_64.egg/deepC/include -isystem /opt/tljh/user/lib/python3.7/site-packages/deepC-0.13-py3.7-linux-x86_64.egg/deepC/packages/eigen-eigen-323c052e1731 gestureTinyModel_deepC/gestureTinyModel.cpp -o gestureTinyModel_deepC/gestureTinyModel.exe
Model executable  gestureTiny

# Gesture guess

In [32]:
#input_file = open(r'https://cainvas-static.s3.amazonaws.com/media/user_data/AyishaR0/test4.txt')
#lines = input_file.readlines()

lines = urllib.request.urlopen('https://cainvas-static.s3.amazonaws.com/media/user_data/AyishaR0/test3.txt').read()
lines = lines.decode('utf-8').split('\n')[:-1]
 
appliance_bool = True
operation_bool = False

# mapping each appliance to the operations that can be done on them
 
appliance_to_operation = {'light' : {'switch'}, 
                          'fan' : {'switch', 'increase', 'decrease'}}
 
# descriptions and cuurent state of the configured (installed) appliances  
configured = {}
configured['light'] = { 1 : ["off"], 
                       2 : ["off"]}
configured['fan'] = { 1 : ["off", 0, 5]}        # 0 - cuurent speed, 5 - maximum

# list of operations performed (just for reference) 
performed = []

# initializations 
count = 0
appliance = ""
 
for line in lines:      # first line of text file contains column headings
 
    line = line.split(' ')[:-1]     # last value is gesture name which is blank here
    line = np.array(line).astype('float32').reshape(1,600)

    result = modelload.predict(normalize(line))
    m = max(result[0])
    g = gestures[np.where(result[0] == m)[0][0]]
    print("Recognized: " + g + " with " + str(m) + " confidence level.")
    
    # if appliance is to be selected
    if (appliance_bool):
        
        if g in configured:
            print("Selected " + g)
            
            # initializing instance of appliance
            count = 0

            # Setting appliance selection to false and operation on appliance to true
            appliance_bool = False
            operation_bool = True
            
            # keeping track of current appliance
            appliance = g
            
            performed.append([g, "Selected"])
        
        else:
            print("Invalid gesture. Try again!")
    
    # if operation is to be done on appliance
    elif (operation_bool):

        # gesture to move to next instance of appliance
        if g == 'countIncrease':
            if count < len(configured[appliance]):
                count = count + 1
                print("Instance " + str(count) + " of " + appliance + " selected.")
                performed.append([appliance, g, count])
            else:
                print("Last instance reached.")

        # gesture to move to previous instance of appliance
        elif g == 'countDecrease':
            if count > 1:
                count = count - 1
                print("Instance " + str(count) + " of " + appliance + " selected.")
                performed.append([appliance, g, count])
            else:
                print("First instance reached.")

        # gesture to change current selelcted appliance
        elif g == 'next':            
            print("Change appliance")
            application = ""

            # Setting appliance selection to true and operation on appliance to false
            appliance_bool = True
            operation_bool = False  

            performed.append(['Changed'])
 
        # gesture to perform an operation on the selelcted appliance (checking if the operation is valid for the selected appliance)
        elif g in appliance_to_operation[appliance]:
            
            # checking if instance of appliance is selected
            if count !=0:
                
                # toggle switch on/off
                if g == 'switch':
                    configured[appliance][count][0] = 'on' if configured[appliance][count][0]=='off' else 'off'

                    performed.append([appliance, g, configured[appliance][count][0]])

                    print("Instance " + str(count) + " of " + appliance + " switched " + configured[appliance][count][0])

                # increase fan speed    
                elif g == 'increase':
                    if configured[appliance][count][1]<configured[appliance][count][2]:
                        configured[appliance][count][1] = configured[appliance][count][1] + 1

                        performed.append([appliance, g, configured[appliance][count][1]])

                        print("Instance " + str(count) + " of " + appliance + " - speed increased to " + str(configured[appliance][count][1]))
                    else:
                        print("Max speed reached!")

                #decrease fan speed
                elif g == 'decrease':
                    if configured[appliance][count][1] > 0:
                        configured[appliance][count][1] = configured[appliance][count][1] - 1

                        performed.append([appliance, g, configured[appliance][count][1]])

                        print("Instance " + str(count) + " of " + appliance + " - speed decreased to " + str(configured[appliance][count][1]))
                    else:
                        print("Min speed reached!")
            else:
                print('Select instance of appliance to work with!')
        else:
            print("Invalid operation for appliance!")
    print()
 
print('\nOperations performed')
for x in performed:
    print(x)
 
print('\nFinal state of appliances')
print(configured)

Recognized: light with 0.90596277 confidence level.
Selected light

Recognized: light with 0.60554725 confidence level.
Invalid operation for appliance!

Recognized: light with 0.5732809 confidence level.
Invalid operation for appliance!

Recognized: countIncrease with 0.999795 confidence level.
Instance 1 of light selected.

Recognized: increase with 0.9759626 confidence level.
Invalid operation for appliance!

Recognized: switch with 0.9970079 confidence level.
Instance 1 of light switched on

Recognized: countIncrease with 0.99781185 confidence level.
Instance 2 of light selected.

Recognized: switch with 0.99973875 confidence level.
Instance 2 of light switched on

Recognized: countDecrease with 0.9991779 confidence level.
Instance 1 of light selected.

Recognized: switch with 0.99664354 confidence level.
Instance 1 of light switched off

Recognized: next with 0.9861981 confidence level.
Change appliance

Recognized: fan with 0.9652178 confidence level.
Selected fan

Recognized: co